In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
try:
    import google.colab
    colab = True
except:
    colab = False

In [ ]:
if colab is True:
    # Running in Google Colab
    # Clone the repo
    !git clone https://github.com/sicara/easy-few-shot-learning
    %cd easy-few-shot-learning
    !pip install .
else:
    # Run locally
    # Ensure working directory is the project's root
    # Make sure easyfsl is installed!
    %cd ..

## Download or Upload Dataset

In [ ]:
# Install gdown if not already installed
!pip install gdown

import gdown
import zipfile
import os

# Google Drive file ID
file_id = "1bcgqIx48jbdNDS1JHySrzw1mzP1qW452"
destination = "/content/BanglaLekha_Isolated_FSL.zip"

# Construct the download URL
download_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
gdown.download(download_url, destination, quiet=False)

# Unzip the file
unzip_dir = "/content/data"
os.makedirs(unzip_dir, exist_ok=True)
with zipfile.ZipFile(destination, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

In [ ]:
# Install gdown if not already installed
!pip install gdown

import gdown
import zipfile
import os

# Google Drive file ID
file_id = "17DPq24559au2vEpH1QeAIL4FWs2jzr09"
destination = "/content/BasicFinalDatabase_FSL.zip"

# Construct the download URL
download_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
gdown.download(download_url, destination, quiet=False)

# Unzip the file
unzip_dir = "/content/data"
os.makedirs(unzip_dir, exist_ok=True)
with zipfile.ZipFile(destination, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

In [ ]:
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm
import torchvision
import torch.utils.data

In [ ]:
random_seed = 30
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Episodic Training

In [ ]:
n_way = 50
n_shot = 5
n_query = 10

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_workers = 0

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

class BengaliCharactersDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.samples = []

        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            for img_file in os.listdir(class_dir):
                if img_file.endswith('.png'):
                    self.samples.append((os.path.join(class_dir, img_file), class_name))

        # Debug: Print the first few samples
        print("First few samples:")
        print(self.samples[:5])  # Adjust the number to print more or fewer samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label_str = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        # Convert label from string to integer
        label = self.class_to_idx[label_str]

        if self.transform:
            image = self.transform(image)

        return image, label

    def get_labels(self):
        return [label for _, label in self.samples]


In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

class BengaliCharactersDataset2(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.samples = []

        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            for img_file in os.listdir(class_dir):
                if img_file.endswith('.bmp'):
                    self.samples.append((os.path.join(class_dir, img_file), class_name))

        # Debug: Print the first few samples
        print("First few samples:")
        print(self.samples[:5])  # Adjust the number to print more or fewer samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label_str = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        # Convert label from string to integer
        label = self.class_to_idx[label_str]

        if self.transform:
            image = self.transform(image)

        return image, label

    def get_labels(self):
        return [label for _, label in self.samples]


In [ ]:
image_size = 84  # Adjusted for ResNet, which typically expects larger input sizes

train_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  # Random rotation between -10 and 10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize to [-1, 1]
])

test_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize to [-1, 1]
])

train_set = BengaliCharactersDataset(root_dir='/content/data/BanglaLekha_Isolated_FSL/Train', transform=train_transforms)
test_set = BengaliCharactersDataset(root_dir='/content/data/BanglaLekha_Isolated_FSL/Test', transform=test_transforms)

In [ ]:
from easyfsl.samplers import TaskSampler
from torch.utils.data import DataLoader


n_tasks_per_epoch = 500
n_validation_tasks = 100

val_set = test_set

train_set.get_labels = lambda: [
    instance[1] for instance in train_set
]

val_set.get_labels = lambda: [
    instance[1] for instance in val_set
]

# Those are special batch samplers that sample few-shot classification tasks with a pre-defined shape
train_sampler = TaskSampler(
    train_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_tasks_per_epoch
)
val_sampler = TaskSampler(
    val_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_validation_tasks
)
# Finally, the DataLoader. We customize the collate_fn so that batches are delivered
# in the shape: (support_images, support_labels, query_images, query_labels, class_ids)
train_loader = DataLoader(
    train_set,
    batch_sampler=train_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)
val_loader = DataLoader(
    val_set,
    batch_sampler=val_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

print(len(train_set), len(val_set))

In [ ]:
#Only used this encoder for RelationNetworks few shot algorithm

class CNNEncoder(nn.Module):
    """docstring for ClassName"""
    def __init__(self):
        super(CNNEncoder, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(3,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=1),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU())
        self.layer4 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=1),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU())

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        #out = out.view(out.size(0),-1)
        return out # 64

from easyfsl.methods import RelationNetworks, FewShotClassifier

convolutional_network = CNNEncoder()
few_shot_classifier = RelationNetworks(convolutional_network,feature_dimension = 64).to(DEVICE)

In [ ]:
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter


LOSS_FUNCTION = nn.CrossEntropyLoss()

n_epochs = 30
scheduler_milestones = [120, 160]
scheduler_gamma = 0.1
learning_rate = 1e-2
tb_logs_dir = Path(".")

train_optimizer = SGD(
    few_shot_classifier.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
)
train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))

In [ ]:
def training_epoch(
    model: FewShotClassifier, data_loader: DataLoader, optimizer: Optimizer
):
    all_loss = []
    model.train()
    with tqdm(
        enumerate(data_loader), total=len(data_loader), desc="Training"
    ) as tqdm_train:
        for episode_index, (
            support_images,
            support_labels,
            query_images,
            query_labels,
            _,
        ) in tqdm_train:
            optimizer.zero_grad()
            model.process_support_set(
                support_images.to(DEVICE), support_labels.to(DEVICE)
            )
            classification_scores = model(query_images.to(DEVICE))

            loss = LOSS_FUNCTION(classification_scores, query_labels.to(DEVICE))

            loss.backward()

            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [ ]:
from easyfsl.utils import evaluate

PATH = '/content/1shot_5way_BanglaRelation_BLF.pth'

best_state = few_shot_classifier.state_dict()
best_validation_accuracy = 0.0
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch(few_shot_classifier, train_loader, train_optimizer)
    validation_accuracy = evaluate(
        few_shot_classifier, val_loader, device=DEVICE, tqdm_prefix="Validation"
    )

    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        best_state = few_shot_classifier.state_dict()
        print("Ding ding ding! We found a new best model!")

    tb_writer.add_scalar("Train/loss", average_loss, epoch)
    tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()

    if epoch % 1 == 0 : torch.save(best_state, PATH)

In [ ]:
few_shot_classifier.load_state_dict(torch.load(PATH))

<All keys matched successfully>

**Evaluation**

In [ ]:
n_test_tasks = 100

test_set.get_labels = lambda: [
    instance[1] for instance in test_set
]

test_sampler = TaskSampler(
    test_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_test_tasks
)
test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

In [ ]:
from easyfsl.utils import evaluate
accuracy = evaluate(few_shot_classifier, test_loader, device=DEVICE)
print(f"Average accuracy : {(100 * accuracy):.2f} %")

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
import torch


def evaluate_with_metrics(model, loader, device):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for _, (support_images, support_labels, query_images, query_labels, _) in enumerate(loader):
            model.process_support_set(support_images.to(device), support_labels.to(device))
            outputs = model(query_images.to(device))
            _, preds = torch.max(outputs, 1)
            all_labels.extend(query_labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    precision, recall, f1_score, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
    return precision, recall, f1_score


In [ ]:
precision, recall, f1_score = evaluate_with_metrics(few_shot_classifier, test_loader, DEVICE)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1_score:.2f}")